In [ ]:
import sys

sys.path.append('../..')

In [ ]:
from pathlib import Path

import pycolmap
import numpy as np
import matplotlib.pyplot as plt

from hloc.utils.io import read_image, get_keypoints, get_matches, list_h5_names
from hloc.visualization import plot_images, plot_keypoints, plot_matches

In [ ]:
import logging

logger = logging.getLogger("hloc")
logger.setLevel(logging.ERROR)

In [ ]:
data = Path("../../data/scenes/")

scene = "0025"

model_name = "disk"

images = data / scene / "images"

features = data / scene / "features" / f"{model_name}.h5"
matches = data / scene / "matches" / f"{model_name}.h5"
sparse = data / scene / "sparse" / model_name

In [ ]:
model = pycolmap.Reconstruction(sparse)
print(model.summary())

In [ ]:
# id0, id1 = 3481, 6237
id0, id1 = 5028, 4782

name0 = model.images[id0].name
name1 = model.images[id1].name

print(f"Image {id0}: {name0}")
print(f"Image {id1}: {name1}")

In [ ]:
from hloc import match_features

out = data / "tmp"

if not out.exists():
    out.mkdir()

pairs_path = out / "pairs.txt"
with open(pairs_path, "w") as f:
    f.write(f"{name0} {name1}\n")

In [ ]:
def show_pair(name0, name1):

    kpts0, kpts1 = get_keypoints(features, name0), get_keypoints(features, name1)


    m, sc = get_matches(out / "matches.h5", name0, name1)

    print("#" * 40)
    print("REPORT")
    print("#" * 40)
    print(f"Number of matches: {len(m)}")
    print(f"Min / mean / median / max score: {sc.min():.3f} / {sc.mean():.3f} / {np.median(sc):.3f} / {sc.max():.3f}")

    fig, ax = plt.subplots(figsize=(4, 3))
    ax.hist(sc, bins=50)
    ax.set_xlabel("Score")
    ax.set_ylabel("Count")
    ax.set_title("Match score distribution")

    ax.set_xlim(0, 1)
    plt.show()

    plot_images([read_image(images / name0), read_image(images / name1)], titles=[name0, name1])
    plot_keypoints([kpts0, kpts1], ps=1)
    plot_matches(kpts0[m[:, 0]], kpts1[m[:, 1]], a=0.3)
    plt.show()

In [ ]:
conf = match_features.confs["disk+lightglue"]

conf["model"]['filter_threshold'] = 0.1

match_features.main(
    conf=conf,
    pairs=pairs_path,
    features=features,
    matches=out / "matches.h5",
    overwrite=True,
)

show_pair(name0, name1)

In [ ]:
img_list = list(images.glob("*.jpg"))

N = 50

ids0 = np.random.choice(len(img_list), N)
ids1 = np.random.choice(len(img_list), N)

pairs = list(zip(ids0, ids1))

# filter if id0 == id1
pairs = [p for p in pairs if p[0] != p[1]]

pairs = [(img_list[p[0]].name, img_list[p[1]].name) for p in pairs]

len(pairs)

In [ ]:
with open(pairs_path, "w") as f:
    for name0, name0 in pairs:
        f.write(f"{name0} {name1}\n")

match_features.main(
    conf=conf,
    pairs=pairs_path,
    features=features,
    matches=out / "matches.h5",
    overwrite=True,
)

In [ ]:
for p in list_h5_names(out / "matches.h5"):
    name0, name1 = p.split("/")
    print(f"Image {name0} and {name1}")
    show_pair(name0, name1)